# Ipywidgets (https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html)

In [ ]:
%pip install ipywidgets
%pip install pandas

In [ ]:
import ipywidgets
import pandas as pd

In [ ]:
name = ipywidgets.Text(description='name:')
name

In [ ]:
name.value

# Guests Settings

In [ ]:
class GuestSettings:
    def __init__(self, name: str, side: str, people_invited_count: int, people_arrived_count:int, comments: str):
        self.name = name
        self.side = side
        self.people_invited_count = people_invited_count
        self.people_arrived_count = people_arrived_count
        self.comments = comments


# Guests Setting Vm

In [ ]:
class GuestSettingsVm(ipywidgets.HBox):
    def __init__(self):
        self.guest_name = ipywidgets.Text(
            placeholder='Add Guest...',
            description='Guest Name:'
        )
        self.guest_side = ipywidgets.Dropdown(
            options=['Bride', 'Groom'],
            description='Guest Side:'
        )
        self.people_invited_count = ipywidgets.BoundedIntText(value=1, min=0, max=8, step=1, description='People Invited:')
        self.people_arrived_count = ipywidgets.BoundedIntText(value=0, min=0, max=1, step=1,
                                                              description='People Arrived:')
        self.people_invited_count.observe(self.update_people_arrived_count_max, 'value')
        self.comments = ipywidgets.Textarea(
            placeholder='Comments...',
            description='Comments:',
            disabled=False
        )
        super().__init__(
            [self.guest_name, self.guest_side, self.people_invited_count, self.people_arrived_count, self.comments])

    def update_people_arrived_count_max(self, *args, **kwargs):
        self.people_arrived_count.max = self.people_invited_count.value

    @property
    def guest_settings(self) -> GuestSettings:
        return GuestSettings(name=self.guest_name.value, side=self.guest_side.value,
                             people_invited_count=self.people_invited_count.value,
                             people_arrived_count=self.people_arrived_count.value,
                             comments=self.comments.value)
    
GuestSettingsVm()

# RSVP Application

In [ ]:
class RSVP(ipywidgets.VBox):
    def __init__(self):
        self.guests_setting_vms = ipywidgets.VBox([])
        self.add_guests_button = ipywidgets.interactive(self.add_guest, {'manual': True, 'manual_name': 'Add Guest'})
        super().__init__([self.add_guests_button, self.guests_setting_vms])
        
    def add_guest(self, *args, **kwargs):
        guest_setting_list = list(self.guests_setting_vms.children)
        guest_setting_list.append(GuestSettingsVm())
        self.guests_setting_vms.children = tuple(guest_setting_list)
        
    @property
    def guests(self) -> pd.DataFrame:
        return pd.DataFrame([o.guest_settings.__dict__ for o in self.guests_setting_vms.children])
    
rsvp = RSVP()
rsvp

## RSVP Guests

In [ ]:
rsvp.guests

# Ipyaggrid

In [ ]:
%pip install ipyaggrid

In [ ]:
import ipyaggrid

## Grid Settings ( https://dgothrek.gitlab.io/ipyaggrid/guide/create.html#sample-setup )

In [ ]:
column_defs = [{'headerName':'Name','field':'name'},
               {'headerName':'Side','field':'side'},
               {'headerName':'Pepole Invited','field':'people_invited_count'},
               {'headerName':'Pepole Arrived','field':'people_arrived_count'},
               {'headerName':'Comments','field':'comments'},
              ]

grid_options = {
    'columnDefs' : column_defs,
    'enableSorting': True,
    'enableFilter': True,
}

ipyaggrid.Grid(grid_data=rsvp.guests, grid_options=grid_options,)

# RSVP Application with Grid

In [ ]:
class RSVP(ipywidgets.VBox):
    def __init__(self):
        self.guests_setting_vms = ipywidgets.VBox([])
        self.add_guests_button = ipywidgets.interactive(self.add_guest, {'manual': True, 'manual_name': 'Add Guest'})
        column_defs = [{'headerName':'Name','field':'name'},
                       {'headerName':'Side','field':'side'},
                       {'headerName':'Pepole Invited','field':'people_invited_count'},
                       {'headerName':'Pepole Arrived','field':'people_arrived_count'},
                       {'headerName':'Comments','field':'comments'},
                      ]
        grid_options = {
            'columnDefs' : column_defs,
            'enableSorting': True,
            'enableFilter': True,
        }
        self.guests_grid = ipyaggrid.Grid(grid_data=self.guests, grid_options=grid_options,)
        super().__init__([self.add_guests_button, self.guests_setting_vms, self.guests_grid])
        
    def add_guest(self, *args, **kwargs):
        guest_setting_list = list(self.guests_setting_vms.children)
        guest_setting_list.append(GuestSettingsVm())
        self.guests_setting_vms.children = tuple(guest_setting_list)
        
    @property
    def guests(self) -> pd.DataFrame:
        return pd.DataFrame([o.guest_settings.__dict__ for o in self.guests_setting_vms.children])
    
rsvp = RSVP()
rsvp

# Expand Ipywidgets VBox

In [ ]:
from typing import List, Callable


class VBoxObservable(ipywidgets.VBox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.observers: List[Callable] = list()
        self.make_observe_vbox_observable_and_trigger_value_change()

    def add_observer(self, observer: Callable):
        self.observers.append(observer)

    def add_observe_to_internal_box_children(self, box: ipywidgets.Box):
        for item in box.children:
            if len(getattr(item, 'children', [])) > 0:
                self.add_observe_to_internal_box_children(item)
            else:
                item.observe(self.on_value_change, names='value')

    def make_observe_vbox_observable_and_trigger_value_change(self):
        self.add_observe_to_internal_box_children(self)
        self.on_value_change()

    def on_value_change(self, *args, **kwargs):
        for observer in self.observers:
            observer(*args, **kwargs)

# RSVP Application with VBoxObserverable

In [ ]:
class RSVP(ipywidgets.VBox):
    def __init__(self):
        self.guests_setting_vms = VBoxObservable([])
        self.guests_setting_vms.add_observer(self.on_guests_settings_vm_change)
        self.add_guests_button = ipywidgets.interactive(self.add_guest, {'manual': True, 'manual_name': 'Add Guest'})
        column_defs = [{'headerName': 'Name', 'field': 'name'},
                       {'headerName': 'Side', 'field': 'side'},
                    {'headerName': 'People Invited', 'field': 'people_invited_count'},
                       {'headerName': 'People Arrived', 'field': 'people_arrived_count'},
                       {'headerName': 'Comments', 'field': 'comments'},
                       ]
        grid_options = {
            'columnDefs': column_defs,
            'enableSorting': True,
            'enableFilter': True,
            'enableEditing': True,
        }
        self.guests_grid = ipyaggrid.Grid(grid_data=self.guests, grid_options=grid_options, )
        super().__init__([self.add_guests_button, self.guests_setting_vms, self.guests_grid])

    def on_guests_settings_vm_change(self, *args, **kwargs):
        self.guests_grid.update_grid_data(self.guests)

    def add_guest(self, *args, **kwargs):
        guest_setting_list = list(self.guests_setting_vms.children)
        guest_setting_list.append(GuestSettingsVm())
        self.guests_setting_vms.children = tuple(guest_setting_list)
        self.guests_setting_vms.make_observe_vbox_observable_and_trigger_value_change()

    @property
    def guests(self):
        return pd.DataFrame([o.guest_settings.__dict__ for o in self.guests_setting_vms.children])

    
rsvp = RSVP()
rsvp

In [ ]:
rsvp.guests